In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
rows = 8
cols = 3
fig, ax = plt.subplots(rows, cols, figsize=(25,25))
fig.suptitle("Showing one random image from each class", y=1.05, fontsize=60) # Adding  y=1.05, fontsize=24 helped me fix the suptitle overlapping with axes issue
data_dir = "../input/pytorch-challange-flower-dataset/dataset/train"
plant_sorted = sorted(os.listdir(data_dir))
plant_id = 0
for i in range(rows):
  for j in range(cols):
    try:
      plant_selected = plant_sorted[plant_id] 
      plant_id += 1
    except:
      break
    plant_selected_images = os.listdir(os.path.join(data_dir,plant_selected)) # returns the list of all files present in each plant category
    plant_selected_random = np.random.choice(plant_selected_images) # picks one plant item from the list as choice, takes a list and returns one random item
    img = plt.imread(os.path.join(data_dir,plant_selected, plant_selected_random))
    ax[i][j].imshow(img)
    ax[i][j].set_title(plant_selected, pad = 10 , fontsize=24)
    
plt.setp(ax, xticks=[],yticks=[])
plt.tight_layout()

In [ ]:
# Check how many files are in the train folder
train_files = sum([len(files) for i, j, files in os.walk("../input/pytorch-challange-flower-dataset/dataset/train")])
print("Total number of samples in train folder")
print(train_files)

# Check how many files are in the test folder
test_files = sum([len(files) for i, j, files in os.walk("../input/pytorch-challange-flower-dataset/dataset/valid")])
print("Total number of samples in test folder")
print(test_files)

# Check how many files are in the validation folder
validation_files = sum([len(files) for i, j, files in os.walk("../input/pytorch-challange-flower-dataset/dataset/valid")])
print("Total number of samples in validation folder")
print(validation_files)

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2

from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.applications.inception_v3 import preprocess_input

import cv2
import os
import random
import collections
from collections import defaultdict

from shutil import copy
from shutil import copytree, rmtree

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as img

import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,Conv2D,Add
from tensorflow.keras.layers import SeparableConv2D,ReLU
from tensorflow.keras.layers import BatchNormalization,MaxPool2D
from tensorflow.keras.layers import GlobalAvgPool2D
from tensorflow.keras import Model
%matplotlib inline

In [ ]:
def train_model(n_classes,num_epochs, nb_train_samples,nb_validation_samples,iter,type):
    K.clear_session()

    img_width, img_height = 299, 299
    if (type == 0):
        train_data_dir = '../input/dataset-splited/train-20220413T074749Z-001/train'
        validation_data_dir = '../input/dataset-splited/test-20220413T074851Z-001/test'
    elif(type == 1):
        train_data_dir = '../input/pytorch-challange-flower-dataset/dataset/train'
        validation_data_dir = '../input/pytorch-challange-flower-dataset/dataset/valid'
    batch_size = 16
    bestmodel_path = 'bestmodel_'+str(n_classes)+'class.hdf5'
    trainedmodel_path = 'trainedmodel_'+str(n_classes)+'class.hdf5'
    #history_path = 'history_'+str(n_classes)+'.log'

    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')


    if(iter == 0):
        inception = tf.keras.applications.xception.Xception(weights=None,include_top=False)
    elif(iter == 1):
        inception = tf.keras.applications.InceptionResNetV2(weights=None, include_top=False)
    elif(iter == 2):
        inception = InceptionV3(weights='imagenet', include_top=False)
    elif(iter == 3):
        inception = tf.keras.applications.densenet.DenseNet201(weights=None, include_top=False)
    elif(iter == 4):
        inception = tf.keras.applications.mobilenet_v2.MobileNetV2( weights=None,include_top=False)
    elif(iter == 5):
        inception = tf.keras.applications.vgg19.VGG19(weights=None, include_top=False)
    elif(iter == 6):
        inception = tf.keras.applications.vgg16.VGG16(weights=None, include_top=False)



    x = inception.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128,activation='relu')(x)
    x = Dropout(0.2)(x)

    predictions = Dense(n_classes,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)
    #callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=20)
    model = Model(inputs=inception.input, outputs=predictions)
    model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    #checkpoint = ModelCheckpoint(filepath=bestmodel_path, verbose=1, save_best_only=True)
    #csv_logger = CSVLogger(history_path)

    history = model.fit(train_generator,
                        steps_per_epoch = nb_train_samples // batch_size,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size,
                        epochs=num_epochs,
                        verbose=1)

    #model.save(trainedmodel_path)
    model.save("my_model")
    class_map = train_generator.class_indices
    return history, class_map

In [ ]:
n_classes = 23
epochs = 200
nb_train_samples = train_files
nb_validation_samples = test_files
history, class_map_3 = train_model(n_classes,epochs, nb_train_samples,nb_validation_samples,2,0)
print(class_map_3)
print(history)